## 텍스트에서 단어 목록 추출하기 및 본문에서 단어 키워드 검색하기
- 분석 대상 문서 파일 읽기
- 문서 정제 실시
- 단어 목록 생성 및 파일 저장

#### Process
1. 텍스트 파일(.txt)이나 엑셀, csv 등의 데이터베이스 파일을 읽는다.
1. 문서 정제 작업
   - Clean texts: 텍스트에서 불필요한 부분 삭제(예: 분석 대상인 본문 이외의 header, footer 등, 구두점, 기호 등)
   - Normalize texts: 텍스트에 나타나는 변이형들의 통일(예: 대소문자 통일, 스펠링 차이 통일 등)
   - Tag texts: 단어의 품사 정보 부착(품사 구분이 필요한 경우에 한해)
   - Split texts: 문서를 문장으로, 문장을 단어로 분리
2. 단어 분리 및 갯수 세기 

#### 1. 텍스트의 단어 목록과 빈도 정보 출력하기

In [3]:
txt_sample = "To Sherlock Holmes she is always the woman. I have seldom heard him mention her under any other name. \
In his eyes she eclipses and predominates the whole of her sex. It was not that he felt any emotion akin to love for Irene Adler. \
All emotions, and that one particularly, were abhorrent to his cold, precise but admirably balanced mind. \
He was, I take it, the most perfect reasoning and observing machine that the world has seen, \
but as a lover he would have placed himself in a false position. He never spoke of the softer passions, save with a gibe and a sneer. \
They were admirable things for the observer—excellent for drawing the veil from men’s motives and actions. \
But for the trained reasoner to admit such intrusions into his own delicate and finely adjusted temperament was to introduce \
a distracting factor which might throw a doubt upon all his mental results. Grit in a sensitive instrument, \
or a crack in one of his own high-power lenses, would not be more disturbing than a strong emotion in a nature such as his. \
And yet there was but one woman to him, and that woman was the late Irene Adler, of dubious and questionable memory."

In [2]:
txt_cleaned = txt_sample.replace('.', '').replace(',', '')
txt_splitted = txt_cleaned.split(' ')

In [ ]:
print(txt_cleaned); print(txt_splitted)

- 단어로 분리된 믄자열 리스트에서 특정 단어의 사용 횟수, 즉 빈도를 확인할 때에는 **count()** 함수를 사용할 수 있다.

In [ ]:
print(txt_splitted.count('a')); print(txt_splitted.count('He'))

- 위의 함수들을 이용하여 *txt_sample*에 들어있는 모든 단어들의 빈도를 구할 수 있다.

In [ ]:
txt_cleaned = txt_sample.replace('.', '').replace(',', '')
txt_splitted = txt_cleaned.split(' ')
unique_words = set(txt_splitted)
# print(unique_words)

for word in list(unique_words)[100:115]:
    print('Frequency of', word , ':', txt_splitted.count(word))

- 그러나 위 출력 결과에서 보듯이 대문자로 시작하는 단어와 소문자로 시작하는 단어가 섞여 있으며 이들은 별개의 다른 단어로 저장되어 있다. 아래에서 'And'와 'and' 등 2개가 다 저장되어 있는지 확인해 보자. 

In [ ]:
print('And' in list(unique_words)); print('and' in list(unique_words))

In [ ]:
txt_cleaned = txt_sample.replace('.', '').replace(',', '')
txt_normed = txt_cleaned.lower()
txt_splitted = txt_normed.split(' ')
unique_words = set(txt_splitted)

for word in list(unique_words)[:10]:
    print('Frequency of', word , ':', txt_splitted.count(word))

#### 2. 단어 목록과 빈도를 구한 후 빈도 목록 데이터프레임 만들기 

In [ ]:
txt_cleaned = txt_sample.replace('.', '').replace(',', '')
txt_normed = txt_cleaned.lower()
txt_splitted = txt_normed.split(' ')
unique_words = set(txt_splitted)

freqs = {}
for word in list(unique_words):
    freqs[word] = txt_splitted.count(word)

print(freqs)

- 빈도 정보는 **collections** 라이브러리의 **Counter** 모듈을 이용하여 구할 수도 있다.

In [ ]:
from collections import Counter

txt_cleaned = txt_sample.replace('.', '').replace(',', '')
txt_normed = txt_cleaned.lower()
txt_splitted = txt_normed.split(' ')

freqs = Counter(txt_splitted)
print(freqs)

- 딕셔너리로 저장된 빈도 정보를 데이터프레임으로 변환하여 엑셀 파일로 내보내기

In [ ]:
print(freqs.items())

In [18]:
import pandas as pd

freq_df = pd.DataFrame(freqs.items(), columns=['word', 'count'])
freq_df = freq_df.sort_values(by=['count'], ascending=False).reset_index(drop=True)

In [ ]:
pd.concat([freq_df.head(), freq_df.tail()])

#### 3. \<The Adventure of Sherlock Holmes\> 데이터 읽어와서 내용 살펴보기(https://www.gutenberg.org/cache/epub/1661/pg1661-images.html)

- 일반 텍스트 파일(.txt)을 읽어와서 단어 빈도 목록 만들기

In [ ]:
with open(file='../data/sherlock_holmes_doyle.txt', mode='r') as f:
    advan_holmes = f.read().replace('\ufeff', '').strip().splitlines()

advan_holmes = advan_holmes[53:11951]
advan_holmes[:5]

In [29]:
import re
from collections import Counter
import pandas as pd

words_list = []
for line in advan_holmes:
    if line != '':
        line_lower = line.lower()
        line_lower = re.sub(pattern='[\.\!\?,\'\"“”]', repl='', string=line_lower)
        line_splitted = line_lower.split(' ')
        words_list.extend(line_splitted)

words_freqs = Counter(words_list)

freq_df = pd.DataFrame(words_freqs.items(), columns=['word', 'count'])
freq_df = freq_df.sort_values(by=['count'], ascending=False).reset_index(drop=True)

freq_df.to_excel('../result/word_freq_sherlock_holmes.xlsx')

- **append()**와 **extend()**의 비교

In [ ]:
x = ['Google', 'Facebook', 'Twitter']
y = ['Instagram', 'TikTok']
x.append(y)
print('Appended x:', x)

x = ['Google', 'Facebook', 'Twitter']
y = ['Instagram', 'TikTok']
x.extend(y)
print('Extended x:', x)

In [ ]:
freq_df[:10]

#### 4. 데이터프레임 형식으로 저장된 csv 파일을 읽어와서 단어 빈도 목록 만들기

In [ ]:
import pandas as pd

sherlock_df = pd.read_csv('../result/sherlock_holmes_updated.csv')
sherlock_df

In [ ]:
sherlock_body = sherlock_df['body'].to_list()
len(sherlock_body)

In [ ]:
from collections import Counter

word_lists = []
for body in sherlock_body:
    bdy = body.lower()
    bdy = re.sub(pattern='[\.\!\?,\'\"“”]', repl='', string=bdy)
    # bdy_splitted = bdy.split(' ')
    bdy_splitted = re.split(' |<p>', bdy)
    word_lists.extend(bdy_splitted)

words_freqs = Counter(word_lists)

for itm in list(words_freqs.items())[:10]:
    print(itm)

In [ ]:
freqs_per_chapter = []
for i, row in sherlock_df.iterrows():
    bdy = row['body'].lower()
    bdy = re.sub(pattern='[\.\!\?,\'\"“”]', repl='', string=bdy)
    # bdy_splitted = bdy.split(' ')
    bdy_splitted = re.split(' |<p>', bdy)
    freq = Counter(bdy_splitted)
    freqs_per_chapter.append(freq)

len(freqs_per_chapter)

In [ ]:
list(freqs_per_chapter[0].items())[:10]